# Eddy ID and tracking on all avalibale 2D ICON EERIE data

This notebook uses the "updated" version of py-eedy-tracker that was improved by Aaron Wienkers during the first eerie hackathon (Nov 2023). Link to the py-eddy-tracker updated version and how to use it - https://github.com/eerie-project/EERIE_hackathon_2023/tree/main/RESULTS/pyeddytracker_xarray_dask_parallel. 


Most of the Eddy ID and tracking functions come from py-eddy-tracker (https://py-eddy-tracker.readthedocs.io/en/stable/) and a lot of other eddy ID examples from EERIE first hackathon (Nov 2023) (https://github.com/eerie-project/EERIE_hackathon_2023/tree/main/RESULTS). 

Regridded 0.25deg ICON 1950 run used (because it is regridded, any model should work). Tracks around 2 million eddies in 39 years of data.

Date created: 12.12.2023

Last edited: 22.05.2024

Stella Bērziņa (stella.berzina@usys.ethz.ch)

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
import glob
import intake
from pathlib import Path
import dask
import pandas as pd
dask.config.set({"array.slicing.split_large_chunks": True}) 
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
import sys
from numpy import arange, ones

import matplotlib as mpl
import matplotlib.lines as mlines
from matplotlib.cm import get_cmap
from matplotlib import colors 
from matplotlib.colors import ListedColormap
from matplotlib.figure import Figure

from py_eddy_tracker.observations.network import NetworkObservations
from py_eddy_tracker.observations.observation import EddiesObservations, Table
from py_eddy_tracker.observations.tracking import TrackEddiesObservations
from py_eddy_tracker.dataset.grid import RegularGridDataset
from py_eddy_tracker.featured_tracking.area_tracker import AreaTracker
from py_eddy_tracker.gui import GUI
from py_eddy_tracker.tracking import Correspondances


from datetime import datetime, timedelta
from netCDF4 import Dataset

import io
import os

import warnings
warnings.filterwarnings("ignore")

# from concurrent.futures import ProcessPoolExecutor
# from concurrent.futures import ThreadPoolExecutor

import dask
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

import cartopy.crs as ccrs
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,LatitudeLocator)
import cartopy.feature as cfeature


In [2]:
grid = xr.open_dataset('/pool/data/ICON/grids/public/mpim/0016/icon_grid_0016_R02B09_O.nc')

### Get model data to ID eddies from

In [3]:
cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")
expid = 'eerie-control-1950'
gridspec = 'gr025'
cat_regrid = cat['dkrz.disk.model-output.icon-esm-er'][expid]['ocean'][gridspec]
print(list(cat_regrid))

['2d_daily_mean', '2d_daily_mean_vertical-remap025', '2d_daily_square', '2d_monthly_mean', '2d_monthly_mean_vertical-remap025', '2d_monthly_square', '5lev_daily_mean', 'eddy_monthly_mean']


In [4]:
ds = cat_regrid['2d_daily_mean'].to_dask()
varname = 'ssh'

In [2]:
"""
Relevant if you want to run analysis on a subset of data. I am using all the time steps here.
"""

# # 2022-2032
# ds_subset = ds.sel(time=slice('2032-01-02','2039-12-31'))
# datearr = [pd.Timestamp(t).to_pydatetime() for t in ds_subset.time.values]
# ds_subset

'\nrelevant if you want to run analysis on a subset of data. we are using all the time teps here\n'

# Eddy ID

In [4]:
# Directories
scratch = '/scratch/b/b382618/try/'

datadir = scratch+expid+'/'+gridspec+'/'
print(datadir)

eddy_dir = datadir+'/eddyID/'
if not os.path.exists(eddy_dir):
    os.makedirs(eddy_dir)

track_dir = datadir+'/eddyTracks/'
if not os.path.exists(track_dir):
    os.makedirs(track_dir)

/scratch/b/b382618/try/eerie-control-1950/gr025/


### Start SLURMCluster on Levante for Parallel Computing

In [26]:
# define cluster; corresponds to one SLURM job
cluster = SLURMCluster(name='dask-cluster',
                      cores=16,  
                      memory='256GB',
                      processes=16,
                      interface='ib0',
                      queue='compute',
                      account='bk1377',
                      walltime='05:00:00',
                      asynchronous=0,
                      log_directory='/home/b/b382618/.log_trash')

In [27]:
# multiply cluster size by 2; equivalent to 2 SLURM jobs
# this will actually open 2 jobs on squeue

cluster.scale(cores=32)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://136.172.120.76:37223/status,
Dashboard: http://136.172.120.76:37223/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://136.172.120.76:43205,Workers: 0
Dashboard: http://136.172.120.76:37223/status,Total threads: 0
Started: Just now,Total memory: 0 B


##### Eddy ID parameters used by AVISO

In [28]:
wavelength = 700 #choice of spatial cutoff for high pass filter in km
step_ht = 0.002 #intervals to search for closed contours (2mm in this case)
pixel_limit = None  # Min and max pixel count for valid contour
shape_error = 70  # Error max (%) between ratio of circle fit and contour

In [29]:
def detection(ncfile, varname, date): 
    g = RegularGridDataset(None, "lon", "lat", centered=True, nc4file=ncfile)  # NOTE: Using 'None' for the .nc file path then requires specifying directly the netcdf4 variable in memory
    g.add_uv(varname)
    g.bessel_high_filter(varname, wavelength, order=1)
        
    a, c = g.eddy_identification(varname, "u", "v", 
    date,  # Date of identification
    step_ht,  # step between two isolines of detection (m)
    pixel_limit=pixel_limit,  # Min and max pixel count for valid contour
    shape_error=70  # Error max (%) between ratio of circle fit and contour
    )
    return a,c,g


In [30]:
def detection_save_netcdf_output(date, tt):
    # Load data from xarray into netcdf4 type
    da_ssh = ds_subset.isel(time=tt).ssh.drop('time')
    da_ssh.load(scheduler='sync')
    da_netcdf = Dataset('in-mem-file', mode='r', memory=da_ssh.to_netcdf())
    
    print('Identifying daily eddies for '+date.strftime('%Y%m%d'))
    a_filtered, c_filtered, g_filtered = detection(da_netcdf,varname,date)
    with Dataset(date.strftime(eddy_dir+"eddyID_anticyclonic_"+date.strftime('%Y%m%d')+".nc"), "w") as h:
        a_filtered.to_netcdf(h)
    with Dataset(date.strftime(eddy_dir+"eddyID_cyclonic_"+date.strftime('%Y%m%d')+".nc"), "w") as h:
        c_filtered.to_netcdf(h)
    del a_filtered
    del c_filtered
    del g_filtered
    del date

In [31]:
lazy_results = []
for i in range(len(datearr)):
    lazy_result = dask.delayed(detection_save_netcdf_output)(date=datearr[i], tt=int(i))
    lazy_results.append(lazy_result)  
    
futures = dask.compute(*lazy_results)
results = dask.compute(*futures)

In [21]:
client.close()
client.shutdown()

# Eddy tracking

In [5]:
from py_eddy_tracker.featured_tracking.area_tracker import AreaTracker
from py_eddy_tracker.tracking import Correspondances

##### Eddy tracking parameters used by AVISO

In [6]:
nb_obs_min = 10 # minimum of 10 points in track to be considered a long trajectory
raw = False # 
cmin = 0.05 # minimum contour
virtual = 4 # number of consecutive timesteps with missing detection allowed
class_kw = dict(cmin=cmin)
zarr = False

In [7]:
# if the number of cores is increased then not enough memory per worker to deal with 37 years of data

cluster = SLURMCluster(name='dask-cluster',
                      cores=4,
                      memory='512GB',
                      processes=4,
                      interface='ib0',
                      queue='compute',
                      account='bk1377',
                      walltime='06:00:00',
                      asynchronous=0,
                      log_directory='/home/b/b382618/.log_trash')

In [8]:
cluster.scale(cores=4)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://136.172.120.78:8787/status,
Dashboard: http://136.172.120.78:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://136.172.120.78:37905,Workers: 0
Dashboard: http://136.172.120.78:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [9]:
#Functions from eddy-tracking.py (aided by Malcolm Roberts)

def tracking(file_objects, previous_correspondance, eddy_type, zarr=False, nb_obs_min=10, raw=True, cmin=0.05, virtual=4):
    # We run a tracking with a tracker which uses contour overlap, on first time step
    output_dir = os.path.dirname(previous_correspondance)
    class_kw = dict(cmin=cmin)
    if not os.path.isfile(previous_correspondance):
        c = Correspondances(
            datasets=file_objects, class_method=AreaTracker, 
            class_kw=class_kw, virtual=virtual
        )
        c.track()
        c.prepare_merging()
    else:
        c = Correspondances(
            datasets=file_objects, class_method=AreaTracker, 
            class_kw=class_kw, virtual=virtual,
            previous_correspondance=previous_correspondance
        )
        c.track()
        c.prepare_merging()
        c.merge()

    new_correspondance = previous_correspondance[:-3]+'_new.nc'
    with Dataset(new_correspondance, "w") as h:
        c.to_netcdf(h)

    try:
        # test can read new file, and then move to replace old file
        nc = Dataset(new_correspondance, 'r')
        os.rename(new_correspondance, previous_correspondance)
    except:
        raise Exception('Error opening new correspondance file '+new_correspondance)

    write_obs_files(c, raw, output_dir, zarr, eddy_type, nb_obs_min)
    

def write_obs_files(c, raw, output_dir, zarr, eddy_type, nb_obs_min):
    kw_write = dict(path=output_dir, zarr_flag=zarr, sign_type=eddy_type)

    fout = os.path.join(output_dir, eddy_type+'_untracked.nc')
    c.get_unused_data(raw_data=raw).write_file(
        filename=fout
    )

    short_c = c._copy()
    short_c.shorter_than(size_max=nb_obs_min)
    short_track = short_c.merge(raw_data=raw)

    if c.longer_than(size_min=nb_obs_min) is False:
        long_track = short_track.empty_dataset()
    else:
        long_track = c.merge(raw_data=raw)

    # We flag obs
    if c.virtual:
        long_track["virtual"][:] = long_track["time"] == 0
        long_track.normalize_longitude()
        long_track.filled_by_interpolation(long_track["virtual"] == 1)
        short_track["virtual"][:] = short_track["time"] == 0
        short_track.normalize_longitude()
        short_track.filled_by_interpolation(short_track["virtual"] == 1)

    print("Longer track saved have %d obs", c.nb_obs_by_tracks.max())
    print(
        "The mean length is %d observations for long track",
        c.nb_obs_by_tracks.mean(),
    )

    fout = os.path.join(output_dir, eddy_type+'_tracks.nc')
    long_track.write_file(filename=fout)
    fout = os.path.join(output_dir, eddy_type+'_short.nc')
    short_track.write_file(
        #filename="%(path)s/%(sign_type)s_track_too_short.nc", **kw_write
        filename=fout
    )



def tracking_eddytype(eddy_type, track_dir, eddy_dir, zarr, nb_obs_min, raw, cmin):
    previous_correspondance = os.path.join(track_dir, eddy_type+'_correspondance.nc')
    search = os.path.join(eddy_dir+'eddyID_'+eddy_type+'_'+'????????.nc')
    print('search files ',search)
    file_objects = sorted(glob.glob(search))
    tracking(file_objects, previous_correspondance, eddy_type, zarr=zarr, nb_obs_min=nb_obs_min, raw=raw, cmin=cmin)


In [ ]:
lazy_results = []
print('eddy_dir = ', eddy_dir)
print('track_dir = ', track_dir)
for eddy_type in ['cyclonic','anticyclonic']:
    lazy_result = dask.delayed(tracking_eddytype)(track_dir=track_dir,
                                                    eddy_dir=eddy_dir,
                                                    eddy_type=eddy_type,
                                                    zarr=zarr, nb_obs_min=nb_obs_min,
                                                    raw=raw, cmin=cmin)
    lazy_results.append(lazy_result)  

futures = dask.compute(*lazy_results)
results = dask.compute(*futures)

eddy_dir =  /scratch/b/b382618/try/eerie-control-1950/gr025//eddyID/
track_dir =  /scratch/b/b382618/try/eerie-control-1950/gr025//eddyTracks/


In [14]:
client.close()
client.shutdown()

# Subsetting

I do this subsetting because in a lot of analysis you don't need all tracked eddies but only the long-lived ones. and the dataset is significantly smaller then.

I choose to use the py-eddy-tracker file format because it is pretty useful if you want to create some of the plots that the package provides code for. Esentially 

#### Tracked eddies

In [2]:
# this file contains all tracked anticyclonic eddies for 37 years
a = TrackEddiesObservations.load_file((
        "/work/bk1377/b382618/icon_malcom/10d/eerie-control-1950/gr025/eddyTracks/anticyclonic_tracks.nc")
)


In [3]:
c = TrackEddiesObservations.load_file(
    ("/work/bk1377/b382618/icon_malcom/10d/eerie-control-1950/gr025/eddyTracks/cyclonic_tracks.nc")
)

In [5]:
print(a)

    | 55045328 observations from 18993.999305555553 to 32871.99930555555 (13879.0 days, ~3966 obs/day)
    |   Speed area      : 36.36 Mkm²/day
    |   Effective area  : 48.69 Mkm²/day
    ----Distribution in Amplitude:
    |   Amplitude bounds (cm)        0.00      1.00      2.00      3.00      4.00      5.00     10.00    500.00
    |   Percent of eddies         :      29.66     26.30     13.78      8.21      5.22     10.58      6.26
    ----Distribution in Radius:
    |   Speed radius (km)            0.00     15.00     30.00     45.00     60.00     75.00    100.00    200.00   2000.00
    |   Percent of eddies         :       1.22     22.49     29.97     18.86     11.41      9.71      6.20      0.13
    |   Effective radius (km)        0.00     15.00     30.00     45.00     60.00     75.00    100.00    200.00   2000.00
    |   Percent of eddies         :       0.96     17.11     25.34     18.71     12.98     13.19     11.38      0.32
    ----Distribution in Latitude
        Latitude b

In [4]:
"""
you can also open the tracked files as an xarray
"""

# a_xr = xr.open_dataset('/scratch/b/b382618/try/eerie-control-1950/gr025/eddyTracks/anticyclonic_tracks.nc')
# c_xr = xr.open_dataset('/scratch/b/b382618/try/eerie-control-1950/gr025/eddyTracks/cyclonic_tracks.nc')

'\nyou can also open the tracked files as an xarray\n'

#### I downloaded aviso observations from the eerie intake catalog do I could open the output as a py-eddy-tracker file

In [7]:
a_obs = TrackEddiesObservations.load_file((
        "/work/bk1377/b382618/icon_malcom/10d/eerie-control-1950/gr025/aviso/anticyclonic.nc"))


In [8]:
c_obs = TrackEddiesObservations.load_file((
        "/work/bk1377/b382618/icon_malcom/10d/eerie-control-1950/gr025/aviso/cyclonic.nc"))


In [9]:
print(a_obs)

    | 34521490 observations from 15706.0 to 26337.0 (10632.0 days, ~3247 obs/day)
    |   Speed area      : 32.95 Mkm²/day
    |   Effective area  : 45.02 Mkm²/day
    ----Distribution in Amplitude:
    |   Amplitude bounds (cm)        0.00      1.00      2.00      3.00      4.00      5.00     10.00    500.00
    |   Percent of eddies         :      14.24     23.74     15.91     10.63      7.42     17.34     10.73
    ----Distribution in Radius:
    |   Speed radius (km)            0.00     15.00     30.00     45.00     60.00     75.00    100.00    200.00   2000.00
    |   Percent of eddies         :       0.02      8.09     35.82     24.60     13.22     10.72      7.38      0.14
    |   Effective radius (km)        0.00     15.00     30.00     45.00     60.00     75.00    100.00    200.00   2000.00
    |   Percent of eddies         :       0.02      5.66     27.03     23.13     15.07     14.92     13.75      0.43
    ----Distribution in Latitude
        Latitude bounds            -90.

Select only the eddies that have a lifetime >= 16 weeks (so you can compare to Chelton et al. 2016)

In [5]:
a_16 = a.extract_with_length((7 * 16, -1))
c_16 = c.extract_with_length((7 * 16, -1))

In [21]:
a_obs16 = a_obs.extract_with_length((7 * 16, -1))
c_obs16 = c_obs.extract_with_length((7 * 16, -1))

In [14]:
type(a_16)

py_eddy_tracker.observations.tracking.TrackEddiesObservations

Save to a directory of your choice. to_netcdf does not really work although it is a function in py-eddy-tracker.

In [16]:
a_16.write_file("/work/bk1377/b382618/icon_malcom/16w/")

In [17]:
c_16.write_file("/work/bk1377/b382618/icon_malcom/16w/")

In [2]:
a_obs16.write_file("/work/bk1377/b382618/icon_malcom/16w/aviso/")

NameError: name 'a_obs16' is not defined

In [ ]:
c_obs16.write_file("/work/bk1377/b382618/icon_malcom/16w/aviso/")